In [2]:
import pandas as pd
import numpy as np


# Loading the training data

Loading the list of foods and description/ingredients list from the USDA (Source: https://fdc.nal.usda.gov/download-datasets.html)

In [ ]:
branded_food_cols = ["fdc_id","brand_owner","brand_name","ingredients","not_a_significant_source_of","branded_food_category"]

In [ ]:
df_branded_food = pd.read_csv("branded_food.csv", usecols = branded_food_cols, on_bad_lines = 'skip')

<ipython-input-8-82944e97e487>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_branded_food = pd.read_csv("branded_food.csv", usecols = branded_food_cols, on_bad_lines = 'skip')


In [ ]:
df_branded_food.head()

,fdc_id,brand_owner,brand_name,ingredients,not_a_significant_source_of,branded_food_category
0,1105904,Richardson Oilseed Products (US) Limited,NaN,Vegetable Oil,NaN,Oils Edible
1,1105905,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: BEEF STOCK, CONTAINS LESS THAN 2%...",NaN,Herbs/Spices/Extracts
2,1105906,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: CLAM STOCK, POTATOES, CLAMS, CREA...",NaN,Prepared Soups
3,1105907,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: WATER, CREAM, BROCCOLI, CELERY, V...",NaN,Prepared Soups
4,1105908,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: CHICKEN STOCK, CONTAINS LESS THAN...",NaN,Herbs/Spices/Extracts


In [ ]:
df_branded_food.shape

(196802, 6)

Loading the list of foods and their names

# Feature Engineering

## Flagging ingredients

In [ ]:
food_ingredients = df_branded_food['ingredients']

In [ ]:
word_to_search = 'Gluten'

# Create a new column to flag rows containing the word
df_branded_food['is_gluten'] = food_ingredients.str.contains(word_to_search, case=False).astype(bool)


In [ ]:
df_branded_food['is_gluten'].value_counts()

False    186521
True      10281
Name: is_gluten, dtype: int64

In [ ]:
meat_list= ["Beef", "Pork", "Chicken", "Lamb", "Turkey", "Duck", "Goose", "Veal", "Venison", "Bison", "Rabbit", "Sausage", "Salami", "Ham", "Bacon", "Prosciutto", "Pancetta", "Chorizo", "Pepperoni", "Jerky", "Schnitzel", "Steak", "Rib", "Loin", "Chop", "Breast", "Thigh", "Wing", "Leg", "Drumstick", "Shank", "processed meats", "horsemeat", "poultry", "deer", "deli", "fowl", "pigmeat", "cow", "turkeys", "cows", "gravy", "cattle", "snail", "crocodile", "sheep", "sheepmeat", "sirloin"]

In [ ]:
# Should probably split this out for pork / beef because of religious dietary restrictions

words_to_search = meat_list

pattern = '|'.join(words_to_search)

df_branded_food['contains_meat'] = food_ingredients.str.contains(pattern, case=False).astype(bool)

In [ ]:
df_branded_food['contains_meat'].value_counts()

False    141011
True      55791
Name: contains_meat, dtype: int64

In [ ]:
shellfish_food = ["Shrimp", "Crab", "Lobster", "Clam", "Mussel", "Oyster", "Scallop", "Abalone", "Squid", "Octopus", "Cuttlefish", "Snail", "Conch", "Periwinkle", "Whelk", "Cockle", "Razor clam", "Geoduck", "Barnacle", "Sea urchin", "Sea cucumber", "Limpet", "Winkle", "Krill", "Prawn", "Langoustine", "Crawfish", "Crayfish", "Slipper lobster", "Horseshoe crab", "Mantis shrimp", "Sand dollar", "Sea snail", "Sea slug", "Sea squirt", "Trepang (sea cucumber)", "Chiton", "uni", "escargot", "crabmeat",]

In [ ]:
words_to_search = shellfish_food

pattern = '|'.join(words_to_search)

df_branded_food['contains_shellfish'] = food_ingredients.str.contains(pattern, case=False).astype(bool)

<ipython-input-18-3162bfa3cb0c>:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_branded_food['contains_shellfish'] = food_ingredients.str.contains(pattern, case=False).astype(bool)


In [ ]:
df_branded_food['contains_shellfish'].value_counts()

False    192974
True       3828
Name: contains_shellfish, dtype: int64

In [ ]:
nuts_list = ["Almond", "Walnut", "Cashew", "Pistachio", "Pecan", "Hazelnut", "Brazil nut", "Macadamia nut", "Pine nut", "Chestnut", "Peanut"]

In [ ]:
words_to_search = nuts_list

pattern = '|'.join(words_to_search)

df_branded_food['contains_nuts'] = food_ingredients.str.contains(pattern, case=False).astype(bool)

In [ ]:
df_branded_food['contains_nuts'].value_counts()

False    175588
True      21214
Name: contains_nuts, dtype: int64

In [ ]:
diary_list = ["Milk", "Diary", "Butter", "Cheese", "Yogurt", "Cream", "Sour cream", "Buttermilk", "Whey", "Casein", "Caseinate", "Curds", "Ghee", "Lactose", "Milkfat", "Half-and-half", "Cottage cheese", "Ricotta cheese", "Cream cheese", "Mascarpone", "Ice cream", "Frozen yogurt", "Custard", "Pudding", "Cheddar", "Mozzarella", "Parmesan", "Swiss", "Gouda", "Brie", "Feta", "Blue cheese", "Provolone", "Monterey Jack", "Colby", "Pepper Jack", "Ricotta", "Havarti", "Muenster", "Camembert", "Fontina", "Gruyere", "Edam", "Asiago", "Manchego", "Cotija", "Queso fresco", "Queso blanco", "Panela", "Halloumi", "Teleme", "Burrata", "Burrini", "Cottage cheese", "Cream cheese", "Neufchâtel", "Quark", "Fromage blanc", "Chevre (goat cheese)", "Roquefort", "Stilton", "Cambozola", "Gorgonzola", "Taleggio", "Emmental", "Limburger", "Taleggio", "Pecorino Romano", "Pecorino Toscano", "Manouri", "Myzithra", "Kasseri", "Kefalotyri", "Mizithra", "Parmigiano-Reggiano", "Grana Padano", "Ricotta salata", "Saganaki", "Sirene", "Tulum", "Tzfat", "Västerbottenost", "Wensleydale"]

In [ ]:
words_to_search = diary_list

pattern = '|'.join(words_to_search)

df_branded_food['contains_diary'] = food_ingredients.str.contains(pattern, case=False).astype(bool)

<ipython-input-24-64c7f7767a92>:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_branded_food['contains_diary'] = food_ingredients.str.contains(pattern, case=False).astype(bool)


In [ ]:
df_branded_food['contains_diary'].value_counts()

False    124098
True      72704
Name: contains_diary, dtype: int64

In [ ]:
soy_list = ["Soybeans", "Tofu", "Tempeh", "Edamame", "Soy milk", "Soy sauce", "Miso", "Natto", "Soy flour", "soy", "bean curd", "tamari", "shoyu", "Soybean", "Textured vegetable protein"]

In [ ]:
words_to_search = soy_list

pattern = '|'.join(words_to_search)

df_branded_food['contains_soy'] = food_ingredients.str.contains(pattern, case=False).astype(bool)

In [ ]:
egg_list = ["Egg","Eggshell","Mayonnaise","Mayo","Bearnaise","caesar","custard","pudding","Quiche","Soufflé","Meringue","Frittata","Omelette"]

In [ ]:
words_to_search = egg_list

pattern = '|'.join(words_to_search)

df_branded_food['contains_egg'] = food_ingredients.str.contains(pattern, case=False).astype(bool)

In [ ]:
animal_byproducts = ["Gelatin", "Tallow", "Lard", "Rennet", "Isinglass", "Fish oil", "Bone char", "Carmine", "Shellac", "Collagen", "Beeswax", "Lanolin", "lipase", "protease", "amylase", "lecithin", "Stearic acid ", "Oleic acid ", "Glycerides", "diglycerides", "monoglycerides", "suet", "Monostearate", "Polysorbates", "carmine", "Cochineal extract", "Natural red 4", "E120", "Natural red coloring", "Crimson lake", "Red food colorant"]

In [ ]:
# This is a really hard list to nail since things like vitamins don't specify animal source so they could be anti-vegan

words_to_search = animal_byproducts

pattern = '|'.join(words_to_search)

df_branded_food['contains_animal_byproducts'] = food_ingredients.str.contains(pattern, case=False).astype(bool)

## Categorizing by diet

In [ ]:
# Vegan

df_branded_food["is_vegan"] = ~(df_branded_food['contains_meat'] \
                                | df_branded_food['contains_shellfish'] \
                                | df_branded_food['contains_diary'] \
                                | df_branded_food['contains_egg'] \
                                | df_branded_food['contains_animal_byproducts'])

In [ ]:
# Vegetarian

df_branded_food["is_vegetarian"] = ~(df_branded_food['contains_meat'] \
                                | df_branded_food['contains_shellfish'])


In [ ]:
# Pescatarian

df_branded_food["is_pescatarian"] = ~df_branded_food['contains_meat']

In [ ]:
df_branded_food.head()

,fdc_id,brand_owner,brand_name,ingredients,not_a_significant_source_of,branded_food_category,is_gluten,contains_meat,contains_shellfish,contains_nuts,contains_diary,contains_soy,contains_egg,contains_animal_byproducts,is_vegan,is_vegetarian,is_pescatarian
0,1105904,Richardson Oilseed Products (US) Limited,NaN,Vegetable Oil,NaN,Oils Edible,False,False,False,False,False,False,False,False,True,True,True
1,1105905,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: BEEF STOCK, CONTAINS LESS THAN 2%...",NaN,Herbs/Spices/Extracts,False,True,False,False,False,False,False,False,False,False,False
2,1105906,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: CLAM STOCK, POTATOES, CLAMS, CREA...",NaN,Prepared Soups,False,True,True,False,True,True,False,True,False,False,False
3,1105907,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: WATER, CREAM, BROCCOLI, CELERY, V...",NaN,Prepared Soups,False,False,False,False,True,True,False,True,False,True,True
4,1105908,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: CHICKEN STOCK, CONTAINS LESS THAN...",NaN,Herbs/Spices/Extracts,False,True,False,False,False,False,False,False,False,False,False


# Writing to a new csv

In [ ]:
df_branded_food.to_csv('data.csv', index=False)

# Generate Dataset for Common Diets

In [3]:
df = pd.read_csv('data.csv')

<ipython-input-3-6456521db808>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


In [5]:
import random

dietary_restrictions = [
    ("vegetarian", "is_vegetarian", 1),
    ("vegan", "is_vegan", 1),
    ("pescatarian", "is_pescatarian", 1),
    ("cannot have meat", "contains_meat", 0),
    ("lactose intolerant", "contains_diary", 0),
    ("not feeling like soy", "contains_soy", 0),
    ("do not want eggs", "contains_egg", 0),
    ("no animal products", "contains_animal_byproducts", 0),
    ("allergic to nuts", "contains_nuts", 0),
    ("gluten free diet", "is_gluten", 0),
    ("allergic to shellfish", "contains_shellfish", 0)
]

smaller_set = df.sample(n=1000, random_state=42)

diet_list = []
label = []
for _, row in smaller_set.iterrows():
    random_diet = random.choice(dietary_restrictions)
    diet_list.append(random_diet[0])
    label.append("Yes" if row[random_diet[1]] == bool(random_diet[2]) else "No")

smaller_set["Diet"] = diet_list
smaller_set["Answer"] = label

smaller_set.head(2)

,fdc_id,brand_owner,brand_name,ingredients,not_a_significant_source_of,branded_food_category,is_gluten,contains_meat,contains_shellfish,contains_nuts,contains_diary,contains_soy,contains_egg,contains_animal_byproducts,is_vegan,is_vegetarian,is_pescatarian,Diet,Answer
90929,402627,SPARTAN,NaN,"PASTEURIZED MILK, CHEESE CULTURES, SALT, ENZYM...",NaN,Cheese,False,False,False,False,True,True,False,False,False,True,True,gluten free diet,Yes
154852,466675,"Frontiere Natural Meats, LLC.",NaN,"ORGANIC BEEF, ORGANIC BLUE CHEESE (ORGANIC MIL...",NaN,Frozen Patties and Burgers,False,True,False,False,True,True,False,False,False,False,False,pescatarian,No


In [9]:
def generate_prompt(row, diet=None):
    if diet:
        diet_type = diet
    else:
          diet_type = row["Diet"]
    prompt = f"""I am {diet_type}, can I have this dish or not?
    The dish contains ingredients: {row['ingredients']}.
    The food category is: {row['branded_food_category']}."""

    if row['not_a_significant_source_of'] == row['not_a_significant_source_of']:
      prompt += f"Not a significant source of: {row['not_a_significant_source_of']}."
    return prompt

smaller_set['Prompt'] = smaller_set.apply(generate_prompt, axis=1)
common_diet_df = smaller_set[["Diet", "Prompt", "Answer", ]]
common_diet_df.head(2)

,Diet,Prompt,Answer
90929,gluten free diet,"I am gluten free diet, can I have this dish or...",Yes
154852,pescatarian,"I am pescatarian, can I have this dish or not?...",No


Then we download the common_diet_df csv, manually reviewed it and corrected a few samples. For example, almond milk is mistakenly considered not vegan, which is not true. There are only very few mistakes though.

# Generate Dataset for Special Diets

In [10]:
rare_dietary_restr = [
    "Alpha-gal Syndrome",
    "Histamine Intolerance",
    "favism",
    "with Phenylketonuria",
    "sensitive to nickel",
    "not feeling like mushroom",
    "don't like spinach",
    "a buddhist",
    "a muslim",
    'Hindu',
    "on keto",
    "on a paleo diet"
]

rare_samples = []
for _ in range(500):
    diet = random.choice(rare_dietary_restr)
    row = df.sample(n=1).iloc[0]
    prompt = generate_prompt(row, diet=diet)
    rare_samples.append([diet, prompt])

rare_samples_df = pd.DataFrame(rare_samples, columns=["Diet", "Prompt"])

# Save the DataFrame to a CSV file for manual review
csv_file_path = "rare_dietary_samples.csv"
rare_samples_df.to_csv(csv_file_path, index=False)

rare_samples_df.head(2)

,Diet,Prompt
0,on a paleo diet,"I am on a paleo diet, can I have this dish or ..."
1,a buddhist,"I am a buddhist, can I have this dish or not? ..."


Then we download the rare_samples_df csv, provide the prompts to GPT 4.0 in a batch size of 10. As my tier can only feed 40 messages in 3 hours, this process took ~4 hours.

Next, we started the manual labelling process. We first find articles which described each of the diet (those articles are later used for information retrieval), understant the diet, and start correcting the labels provided by GPT 4.

GPT got 418 samples right out of 500. We changed the labels for 82 samples.

# After manual review, we can train the model using it.

In [13]:
df_with_label = pd.read_csv('df_with_labels.csv')
df_with_label.sample(2)

,Answer,Diet,Ingredients,Prompt
984,Yes,do not want eggs,"TOP ROUND BEEF, BROWN SUGAR, GLUTEN FREE REDUC...","The dish contains: TOP ROUND BEEF, BROWN SUGAR..."
1319,Yes,pescatarian,"CARBONATED EUROPEAN SPRING WATER, CANE SUGAR, ...",The dish contains: CARBONATED EUROPEAN SPRING ...


Split & model training is in the other modeling jupyter notebook.

# Information Retrivel as a Pre-Processing

We evaluate on the ones the model falsely identify
(an accuracy chart by diet type + 2 prompt and answer examples).

We feel like for harder diets, like hindu, keto, and special allergies, the model does not fully understand what the diet is. We want to refer to some articles that are about the diet, and use information retrivel model to summarize the diet behavious. Hopefully adding this to prompt would better frame the question & allow the model to have better results.

### Articles

In [14]:
# https://en.wikipedia.org/wiki/List_of_diets

wiki_article = """
An individual's diet is the sum of food and drink that one habitually consumes. Dieting is the practice of attempting to achieve or maintain a certain weight through diet.[1] People's dietary choices are often affected by a variety of factors, including ethical and religious beliefs, clinical need, or a desire to control weight.

Not all diets are considered healthy. Some people follow unhealthy diets through habit, rather than through a conscious choice to eat unhealthily. Terms applied to such eating habits include "junk food diet" and "Western diet". Many diets are considered by clinicians to pose significant health risks and minimal long-term benefit. This is particularly true of "crash" or "fad" diets – short-term, weight-loss plans that involve drastic changes to a person's normal eating habits.

Belief-based diets
Some people's dietary choices are influenced by their religious, spiritual or philosophical beliefs.

Buddhist diet: While Buddhism does not have specific dietary rules, some Buddhists practice vegetarianism based on Mahayana Buddhism's strict interpretation of the first of the Five Precepts.[2]
Hindu diet: It is popular for followers of Hinduism to follow lacto vegetarian diets (though most do not), based on the principle of ahimsa (non-harming).[3] Consuming beef/cattle is forbidden or at least taboo among followers due to cow veneration. Most Hindus in India do intentionally limit their meat consumption one way or another.[4]
Jain diet: Due to how the Jain faith interprets ahisma, vegetarianism is considered mandatory for followers; a lacto-vegetarian diet[5] or vegan diet[6] in particular is considered appropriate for Jains. Most Jains also abstain from consuming root vegetables in order to prevent harming insects, worms and microorganisms when they are uprooted. Most also partake in some form of fasting.[4] Some variants of Jainism further discourage or forbid the consumption of honey, fungi, alcoholic beverages and fermented foods.
Islamic diet: Muslims follow a diet consisting solely of food that is halal – permissible in Islam. The opposite of halal is haraam, food that is Islamically Impermissible. Haraam substances include alcohol, carnivores, pork and other non-ruminant animals, and any meat from an animal which was not killed through the Islamic method of ritual slaughter (Dhabiha).[7] If an otherwise Halal animal was subject to torture by humans, its meat can still be considered non-permissible for Muslims.
I-tal: A set of principles which influences the diet of many members of the Rastafari movement. One principle is that natural foods should be consumed. Emphasis is put on consuming produce that is fresh, organic and ideally grown at home or locally. Another principle involves avoiding "unclean" types of food; the definition which is influenced by Biblical teachings. In order to preserve "life energy" Rastafarians encourage teetotalism, and many Rastafarians interpret I-tal to advocate vegetarianism or veganism as well.[8] Many followers do view seafood as an acceptable addition to an I-tal diet but they restrict which kinds they permit; fish over a foot long are typically avoided and all shellfish are eschewed as they are not Kosher animals—unlike finned-fish with scales.
Kosher diet: Food permissible under Kashrut, the set of Jewish dietary laws, is said to be Kosher. Some foods and food combinations are non-Kosher, and failure to prepare food in accordance with Kashrut can make otherwise permissible foods non-Kosher.[9]
Seventh-day Adventist diet: Combines the Kosher food rules of Judaism with prohibitions against alcoholic beverages and (sometimes) caffeinated beverages. There is emphasis on consuming whole foods. Meat-consumption is heavily discouraged but not necessarily disallowed; about half of Adventists are lacto-ovo-vegetarians.[10] Vegan and pescetarian diets are also more popular among Adventists compared to the general public[11] but other Adventists are still willing to eat Kosher meats.
Word of Wisdom diet: The name of a section of the Doctrine and Covenants, a book of scripture accepted by members of the Church of Jesus Christ of Latter-day Saints. Dietary advice includes (1) wholesome plants "in the season thereof", (2) eating meat sparingly and only "in times of winter, or of cold, or famine", and (3) grain as the "staff of life".[12] Unlike injunctions against tobacco, alcohol, coffee and tea—compliance with meat-avoidance has always remained optional among the Church of Jesus Christ of Latter-day Saints and emphasis on refraining from meat has largely been dropped.[13] An official church publication states, "modern methods of refrigeration now make it possible to preserve meat in any season".[14]
Calorie and weight control diets

Main article: Dieting
A desire to lose weight is a common motivation to change dietary habits, as is a desire to maintain an existing weight. Many weight loss diets are considered by some to entail varying degrees of health risk, and some are not widely considered to be effective. This is especially true of "crash" or "fad" diets.[15]

Many of the diets listed below could fall into more than one subcategory. Where this is the case, it is noted in that diet's entry.

Low-calorie diets: a dietary regimen that reduces the energy intake from foods and beverages without incurring malnutrition.The possible effect of calorie restriction on body weight management, longevity, and aging-associated diseases has been an active area of research.
5:2 diet: an intermittent fasting diet
Intermittent fasting: Cycling between non-fasting and fasting as a method of calorie restriction.[16]
Body for Life: A calorie-control diet, promoted as part of the 12-week Body for Life program.[17]
Cookie diet: A calorie control diet in which low-fat cookies are eaten to quell hunger, often in place of a meal.[18]
The Hacker's Diet: A calorie-control diet from The Hacker's Diet by John Walker. The book suggests that the key to reaching and maintaining the desired weight is understanding and carefully monitoring calories consumed and used.
Nutrisystem diet: The dietary element of the weight-loss plan from Nutrisystem, Inc. Nutrisystem distributes low-calorie meals, with specific ratios of fats, proteins and carbohydrates.[19]
Weight Watchers diet: Foods are assigned point values; dieters can eat any food with a point value provided they stay within their daily point limit.[20]

Very low calorie diets: A very low calorie diet is consuming fewer than 800 calories per day. Such diets are normally followed under the supervision of a doctor. Zero-calorie diets are also included.

Inedia (breatharian diet): A diet in which no food is consumed, based on the belief that prana but not food is necessary for human subsistence.[22]
KE diet (feeding tube diet): A diet in which an individual feeds through a feeding tube and does not eat anything.[23]
The Last Chance diet: General premise is that the dieter will consume only one low-calorie high protein beverage daily. This equated to no more than 400 calories per day.[24][25]
Tongue Patch Diet: Stitching a Marlex patch to the tongue to make eating painful. Daily calories are then limited to 800 per day maximum in liquid form.
Low-carbohydrate diets

Atkins diet: A low-carbohydrate diet, popularized by nutritionist Robert Atkins in the late-20th and early-21st centuries.[26] Proponents argue that this approach is a more successful way of losing weight than low-calorie diets;[27] critics argue that a low-carb approach poses increased health risks.[28] The Atkins diet consists of four phases (Induction, Balancing, Fine-Tuning and Maintenance) with a gradual increase in consumption of carbohydrates as the person goes through the phases.[29]
Dukan Diet: A multi-step diet based on high protein and limited carbohydrate consumption. It starts with two steps intended to facilitate short term weight loss, followed by two steps intended to consolidate these losses and return to a more balanced long-term diet.[30]
Kimkins: A heavily promoted diet for weight loss, found to be fraudulent.
South Beach Diet: Diet developed by the Miami-based cardiologist Arthur Agatston, who says that the key to losing weight quickly and getting healthy is not cutting all carbohydrates and fats from the diet, but choosing the right carbs and the right fats.[31]
Stillman diet: A carbohydrate-restricted diet that predates the Atkins diet, allowing consumption of specific food ingredients.
Low-fat diets
Main article: Low-fat diet
McDougall's starch diet is a high calorie, high fiber, low fat diet that is based on starches such as potatoes, rice, and beans which excludes all animal foods and added vegetable oils. John A. McDougall draws on historical observation of how many civilizations around the world throughout time have thrived on starch foods.
Crash diets
Crash diets are very-low-calorie diets used for the purpose of very fast weight loss.[32][33][34] They describe diet plans that involve making extreme, rapid changes to food consumption, but are also used as disparaging terms for common eating habits which are considered unhealthy. This diet is dangerous and can lead to sudden death when not done in a medically supervised setting.[35][36] Several diets listed here are weight-loss diets which would also fit into other sections of this list. Where this is the case, it will be noted in that diet's entry.

Beverly Hills Diet: An extreme diet which has only fruits in the first days, gradually increasing the selection of foods up to the sixth week.[37]
Cabbage soup diet: A low-calorie diet based on heavy consumption of cabbage soup. Considered a fad diet.[38]
Grapefruit diet: A fad diet, intended to facilitate weight loss, in which grapefruit is consumed in large quantities at meal times.[39]
Monotrophic diet: A diet that involves eating only one food item, or one type of food, for a period of time to achieve a desired weight reduction.
Subway diet: A crash diet[40] in which a person consumes Subway sandwiches in place of higher calorie fast foods. Made famous by convicted sex offender and former obese student Jared Fogle, who lost 245 pounds after replacing his meals with Subway sandwiches as part of an effort to lose weight.[40]
Detox diets
Detox diets involve either not consuming or attempting to flush out substances that are considered unhelpful or harmful. Examples include restricting food consumption to foods without colorings or preservatives, taking supplements, or drinking large amounts of water. The latter practice in particular has drawn criticism, as drinking significantly more water than recommended levels can cause hyponatremia.[41] There is no scientific evidence of any benefit from detox diets, and so they are considered to be pseudoscientific.[42][43]

Juice fasting: A form of detox diet, in which nutrition is obtained solely from fruit and vegetable juices. The health implications of such diets are disputed.[44]
Master Cleanse: A modified juice fast that substitutes tea and lemonade for food.
Diets followed for medical reasons
People's dietary choices are sometimes affected by intolerance or allergy to certain types of food. There are also dietary patterns that might be recommended, prescribed or administered by medical professionals for people with specific medical needs.

DASH diet (Dietary Approaches to Stop Hypertension): A recommendation that those with high blood pressure consume large quantities of fruits, vegetables, whole-grains and low fat dairy foods as part of their diet, and avoid sugar sweetened foods, red meat and fats. Promoted by the US Department of Health and Human Services, a United States government organisation.[45]
Diabetic diet: An umbrella term for diets recommended to people with diabetes. There is considerable disagreement in the scientific community as to what sort of diet is best for people with diabetes.[46]
Elemental diet: A medical, liquid-only diet, in which liquid nutrients are consumed for ease of ingestion.[47]
Elimination diet: A method of identifying foods which cause a person adverse effects, by process of elimination.[48]
Gluten-free diet: A diet which avoids the protein gluten, which is found in barley, rye and wheat. It is a medical treatment for gluten-related disorders, which include coeliac disease, non-celiac gluten sensitivity, gluten ataxia, dermatitis herpetiformis and wheat allergy.[49][50][51][52]
Gluten-free, casein-free diet: A gluten-free diet which also avoids casein, a protein commonly found in milk and cheese. This diet has been researched for efficacy in treatment of autism spectrum disorder.[53]
Healthy kidney diet: This diet is for those impacted with chronic kidney disease, those with only one kidney, those who have a kidney infection and those who may be suffering from some other kidney failure.[54] This diet is not the dialysis diet,[55] which is completely different. The healthy kidney diet restricts large amounts of protein, which are hard for the kidney to break down, but especially limits potassium and phosphorus-rich foods and beverages. Liquid intake is often limited as well.[54][56]
Ketogenic diet (Keto): A high-fat, low-carb diet, in which dietary and body fat is converted into energy. It is used as a medical treatment for refractory epilepsy.[57]
Liquid diet: A diet in which only liquids are consumed. May be administered by clinicians for medical reasons, such as after a gastric bypass[58] or to prevent death through starvation from a hunger strike.[59]
Low-FODMAP diet: A diet that consists in the global restriction of all fermentable carbohydrates (FODMAPs).
Soft diet: A mechanical soft diet or edentulous diet, or soft food(s) diet, is a diet that involves only foods that are physically soft, with the goal of reducing or eliminating the need to chew the food
Specific carbohydrate diet: A diet that aims to restrict the intake of complex carbohydrates such as found in grains and complex sugars.[60]
Fad diets: A fad diet is a diet that is popular for a time, similar to fads in fashion, without being a standard dietary recommendation, and often promising unreasonably fast weight loss or nonsensical health improvements.[61][62][63][64][65] There is no single definition of what a fad diet is, encompassing a variety of diets with different approaches and evidence bases, and thus different outcomes, advantages and disadvantages,[62] and it is ever-changing.[61][62] Generally, fad diets promise short-term changes with little effort, and thus may lack educating consumers about whole-diet, whole lifestyle changes necessary for sustainable health benefits.[61][62][66][67] Fad diets are often promoted with exaggerated claims, such as rapid weight loss of more than 1 kg/week or improving health by "detoxification", or even dangerous claims.[62][63][68][69]
Nickel Allergy: Avoid all foods that are routinely high in nickel content such as cocoa, chocolate, soya beans, oatmeal, nuts, almonds and fresh and dried legumes. Avoid all drinks and vitamin supplements with nickel and canned food.: Avoid all foods that are routinely high in nickel content such as cocoa, chocolate, soya beans, oatmeal, nuts, almonds and fresh and dried legumes. Avoid all drinks and vitamin supplements with nickel and canned food.

Vegetarian diets: A vegetarian diet is one which excludes meat. Vegetarians also avoid food containing by-products of animal slaughter, such as animal-derived rennet and gelatin.[142]

Fruitarian diet: A diet which predominantly consists of raw fruit.[143]
Lacto vegetarianism: A vegetarian diet that includes certain types of dairy, but excludes eggs and foods which contain animal rennet.[144] A common diet among followers of several religions, including Hinduism, Sikhism and Jainism, based on the principle of Ahimsa (non-harming).[3]
Ovo vegetarianism: A vegetarian diet that includes eggs, but excludes dairy.
Ovo-lacto vegetarianism: A vegetarian diet that includes eggs and dairy.[144]
Vegan diet: In addition to the abstentions of a vegetarian diet, vegans do not use any product produced by animals, such as eggs, dairy products, or honey.[142] The vegan philosophy and lifestyle is broader than just the diet and also includes abstaining from using any products tested on animals and often campaigning for animal welfare and animal rights.
Semi-vegetarian diets
Semi-vegetarianism: A predominantly vegetarian diet, in which meat is occasionally consumed. This includes "flexitarian", reducetarian and demitarian diets [145] Sometimes semi-vegetarian and flexitarian diets are defined as distinct from one another, where the former is defined as abstaining from red meat while the latter simply entails only eating meat infrequently.[146][147][148]
Pescetarianism: A diet which includes seafood, but not poultry, other white meat or meat from mammals.
Pollotarianism: A diet which includes poultry, but no other white meat, seafood or meat from mammals.
Kangatarian: A diet originating from Australia. In addition to foods permissible in a vegetarian diet, kangaroo meat is also consumed.[149] The name is a protologism that may have started out as a joke rather than a dietary term or identifying label that was ever intended to be taken seriously or used unironically.[150]
Planetary health diet: Dietary paradigms that have the following aims: to feed a growing world's population, to greatly reduce the worldwide number of deaths caused by poor diet, and to be environmentally sustainable as to prevent the collapse of the natural world.[151]
Plant-based diet: A broad term to describe diets in which animal products do not form a large proportion of the diet. Under some definitions a plant-based diet is fully vegetarian; under others it is possible to follow a plant-based diet whilst occasionally consuming meat.[152]
Other diets
Alkaline diet: The avoidance of relatively acidic foods – foods with low pH levels – such as alcohol, caffeine, dairy, fungi, grains, meat, and sugar. Proponents believe such a diet may have health benefits;[153] critics consider the arguments to have no scientific basis.[154]
Clean eating
Climatarian diet: A diet focused on reducing the carbon footprint of the consumed food, particularly through the consumption of locally sourced food and the avoidance of beef and lamb meat.[155] Adherents may also be "organivores" (strong proponents of certified organic foods over intensively farmed foods).[156]
Eat-clean diet: Focuses on eating foods without preservatives, and on mixing lean proteins with complex carbohydrates.[157]
Gerson therapy: A form of alternative medicine, the diet is low salt, low fat and vegetarian, and also involves taking specific supplements. It was developed by Max Gerson, who claimed the therapy could cure cancer and chronic, degenerative diseases. These claims have not been scientifically proven, and they can cause serious illness and death.[158]
The Graham Diet: A vegetarian diet which promotes whole-wheat flour and discourages the consumption of stimulants such as alcohol and caffeine. Developed by Sylvester Graham in the 19th century.[159]
Hay diet: A food-combining diet developed by William Howard Hay in the 1920s. Divides foods into separate groups, and suggests that proteins and carbohydrates should not be consumed in the same meal.[81]
High-protein diet: A diet in which high quantities of protein are consumed with the intention of building muscle. Not to be confused with low-carb diets, where the intention is to lose weight by restricting carbohydrates.
High residue diet: A diet in which high quantities of dietary fiber are consumed. High-fiber foods include certain fruits, vegetables, nuts and grains.[160]
A group of people in winter clothing, standing around piles of meat lying on the snow.
Sharing of frozen, aged walrus meat among Inuit families
Inuit diet: Inuit traditionally consume food that is fished, hunted or gathered locally, predominantly meat and fish.[161]
Jenny Craig: A weight-loss program from Jenny Craig, Inc. It includes weight counselling among other elements. The dietary aspect involves the consumption of pre-packaged food produced by the company.[162]
Locavore diet: a neologism describing the eating of food that is locally produced, and not moved long distances to market. An example of this was explored in the book 100-Mile Diet, in which the authors only consumed food grown within 100 miles of their residence for a year.[163] People who follow this type of diet are sometimes known as locavores.
Low carbon diet: Consuming food which has been produced, prepared and transported with a minimum of associated greenhouse gas emissions.

An assortment of foods on a worktop. They are: tofu, wheat miso, bancha tea, umeboshi prumes brown rice, sea salt and nori.
Some common macrobiotic ingredients
Macrobiotic diet: A diet in which processed food is avoided. Common components include grains, beans and vegetables.[164]
Mediterranean diet: A diet based on habits of some southern European countries. One of the more distinct features is that olive oil is used as the primary source of fat.[165]
MIND diet: combines the portions of the DASH diet and the Mediterranean diet. The diet is intended to reduce neurological deterioration such as Alzheimer's disease.[166]
Montignac diet: A weight-loss diet characterised by consuming carbohydrates with a low glycemic index.[167]
Mushroom diet: A mushroom-predominant diet.
Negative calorie diet: A claim by many weight-loss diets that some foods take more calories to digest than they provide, such as celery. The basis for this claim is disputed.[168]
Okinawa diet: A low-calorie diet based on the traditional eating habits of people from the Ryukyu Islands.
Omnivorous diet: An omnivore consumes diverse range of both plant and animal-based food.[169]
Organic food diet: A diet consisting only of food which is organic – it has not been produced with modern inputs such as synthetic fertilizers, genetic modification, irradiation, antibiotics, growth hormones, or synthetic food additives.[170]
Prison loaf: A meal replacement served in some United States prisons to inmates who are not trusted to use cutlery. Its composition varies between institutions and states, but as a replacement for standard food, it is intended to provide inmates with all their dietary needs.
Raw foodism: A diet which centres on the consumption of uncooked and unprocessed food. Often associated with a vegetarian diet,[171] although some raw food dieters do consume raw meat.[172]
Shangri-La Diet
Slimming World diet
Slow-carb diet
Smart For Life
Sonoma diet: A diet based on portion control and centered around consuming "power foods"
SparkPeople diet
Sugar Busters!: Focuses on restricting the consumption of refined carbohydrates, particularly sugars.[173]
Western pattern diet (WPD): 'Default' diet in many developed countries, especially the Anglosphere. The name is from "Western world" and is interchanged with "standard American diet" and "meat-sweet diet" due to the high amount of meat (total), red meats (particularly), dairy, sweets and refined cereals.[174] Subpar intake of whole grains, legumes, tree nuts, produce and seafood is the norm.[175][176] WPDs are distinguished from other unbalanced diets by heavy inclusion of 'junk food' and other ultra-processed foods that generally provide substantial empty calories, carbohydrates, saturated fat, industrial trans fat, added sugar/free sugars, added salt, artificial flavor/sweetener and other processing ingredients. Archetypal examples include: RTE cereals, white breads, fast food, other convenience meals, cured meat dishes, smoked/fried meats, fried dough foods, shallow/deep fried potatoes, other foods intensely fried in rendered fat/refined oil, sugary/fatty discretionary foods (e.g., sauce, candy), colas and other sweetened soft drinks

"""

mayo_clinic_articles = """
People with lactose intolerance are unable to fully digest the sugar (lactose) in milk.
Histamine intolerance is an inability to break down histamine in your body, causing it to build up. Certain foods that are high in histamine or that cause your body to release histamine can give you an upset stomach, headache or allergy symptoms. Common trigger foods include fish, alcohol, and fermented or aged foods.
Alpha-gal syndrome is a type of food allergy. It makes people allergic to red meat and other products made from mammals. In the United States, the condition usually begins with the bite of the Lone Star tick.
Phenylketonuria, also called PKU, is a rare inherited disorder that causes an amino acid called phenylalanine to build up in the body. PKU is caused by a change in the phenylalanine hydroxylase (PAH) gene. A dangerous buildup can develop when a person with Phenylketonuria eats foods that contain protein or eats aspartame, an artificial sweetener. This can eventually lead to serious health problems. For the rest of their lives, people with PKU — babies, children and adults — need to follow a diet that limits phenylalanine, which is found mostly in foods that contain protein.
Favism is an acute hemolytic syndrome that occurs in patients with glucose-6-phosphate dehydrogenase (G6PD) deficiency following the ingestion of fava beans. Like the name of Favism, Fava beans is the hallmark trigger for G6PD mutation carriers, other triggers include certain medicines including aspirin, quinine and other antimalarials derived from quinine.
A paleo diet is an eating plan based on foods humans might have eaten during the Paleolithic Era. A modern paleo diet includes fruits, vegetables, lean meats, fish, eggs, nuts and seeds. These are foods that in the past people could get by hunting and gathering.
Nickel allergy is a common cause of allergic contact dermatitis. Avoid all foods that are routinely high in nickel content such as cocoa, chocolate, soya beans, oatmeal, nuts, almonds and fresh and dried legumes. Avoid all drinks and vitamin supplements with nickel and canned food.
"""

### Code

In [ ]:
!pip install InstructorEmbedding
!pip install sentence_transformers==2.2.2

In [19]:
from InstructorEmbedding import INSTRUCTOR
embed_model = INSTRUCTOR('hkunlp/instructor-large')

from sklearn.metrics.pairwise import cosine_similarity

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [22]:
corpus_1 = 'Represent the Wikipedia document for retrieval: '
corpus_2_list = (wiki_article + mayo_clinic_articles).split("\n")
corpus_2_list.remove('')
corpus = [[corpus_1, x]for x in corpus_2_list]
corpus_embeddings = embed_model.encode(corpus)

diet_summary = {}
for diet_type in df_with_label.Diet.unique():
  query  = [["Represent the Wikipedia question for retrieving supporting documents: ",f"where does it mean by {diet_type} diet"]]
  query_embeddings = embed_model.encode(query)
  similarities = cosine_similarity(query_embeddings,corpus_embeddings)
  retrieved_doc_id = np.argmax(similarities)
  if np.max(similarities) > 0.9:
    diet_summary[diet_type] = corpus_2_list[retrieved_doc_id]
    print(diet_type, np.max(similarities), corpus_2_list[retrieved_doc_id])
  else:
    print(diet_type, np.max(similarities), 'None')


do not want eggs 0.87436336 None
allergic to nuts 0.8752918 None
vegan 0.9060992 Vegan diet: In addition to the abstentions of a vegetarian diet, vegans do not use any product produced by animals, such as eggs, dairy products, or honey.[142] The vegan philosophy and lifestyle is broader than just the diet and also includes abstaining from using any products tested on animals and often campaigning for animal welfare and animal rights.
gluten free diet 0.92882353 Gluten-free diet: A diet which avoids the protein gluten, which is found in barley, rye and wheat. It is a medical treatment for gluten-related disorders, which include coeliac disease, non-celiac gluten sensitivity, gluten ataxia, dermatitis herpetiformis and wheat allergy.[49][50][51][52]
no animal products 0.8946518 None
pescatarian 0.9180593 Pescetarianism: A diet which includes seafood, but not poultry, other white meat or meat from mammals.
allergic to shellfish 0.86856395 None
not feeling like soy 0.85239244 None
cannot h

In [27]:
#Append summary to df_with_label
df_with_label['PromptwithIR'] = [f"I am {diet_type}, can I have the dish or not?" for diet_type in df_with_label['Diet']]
for i in range(len(df_with_label)):
  diet_type = df_with_label.loc[i, 'Diet']
  if diet_type in diet_summary.keys():
    df_with_label.loc[i, 'PromptwithIR'] += f' {diet_summary[diet_type]}'
  df_with_label.loc[i, 'PromptwithIR'] += f' The dish contains ingredients: {df_with_label.loc[i, "Ingredients"]}'

df_with_label.to_csv('df_with_ir.csv')

In [28]:
df_with_label["PromptwithIR"].sample(2).values

array(['I am on keto, can I have the dish or not? Ketogenic diet (Keto): A high-fat, low-carb diet, in which dietary and body fat is converted into energy. It is used as a medical treatment for refractory epilepsy.[57] The dish contains ingredients: CHOW MEIN NOODLE (WATER, WHEAT FLOUR, SOYBEAN OIL, SALT, SODIUM CARBONATE, CORNSTARCH.), VEGETABLE MIX (OYSTER MUSHROOMS, BLACK MUSHROOMS, BAMBOO SHOOTS, SHIITAKE MUSHROOMS, CHAMPIGNON MUSHROOMS, ASPARAGUS.), SOY SAUCE (WATER, SOYBEANS, WHEAT.), WATER, SUGAR, SOYBEAN OIL, RICE WINE, GARLIC, SESAME OIL, BLACK PEPPER, SALT.. Category: Frozen Dinners & Entrees',
       'I am vegan, can I have the dish or not? Vegan diet: In addition to the abstentions of a vegetarian diet, vegans do not use any product produced by animals, such as eggs, dairy products, or honey.[142] The vegan philosophy and lifestyle is broader than just the diet and also includes abstaining from using any products tested on animals and often campaigning for animal welfare an

# End of Data Wrangling

Now we have two datasets: df_with_label, which is the base dataset, and df_with_ir, which is the dataset with information retrieval attempt in the prompt.